<a href="https://colab.research.google.com/github/Sarah0711/CUDA-Vision-Lab/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
if not os.path.exists('data.zip'): 
  !wget --no-check-certificate !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qrP1Qxrn1LztYOa-qQh_NA44YTGZyZ29' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1qrP1Qxrn1LztYOa-qQh_NA44YTGZyZ29" -O  data.zip && rm -rf /tmp/cookies.txt

--2020-02-15 19:34:57--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2020-02-15 19:34:57--  https://docs.google.com/uc?export=download&confirm=ezc9&id=1qrP1Qxrn1LztYOa-qQh_NA44YTGZyZ29
Resolving docs.google.com (docs.google.com)... 74.125.24.102, 74.125.24.101, 74.125.24.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-3o-docs.googleusercontent.com/docs/securesc/thvgrdqa8tpdto4uj02l91ng39l87h8o/fou7dgoi7m0uqhmrhe44me467dbpi1s1/1581795000000/02751230155057180797/12997425790566882683Z/1qrP1Qxrn1LztYOa-qQh_NA44YTGZyZ29?e=download [following]
--2020-02-15 19:34:58--  https://doc-0g-3o-docs.googleusercontent.com/docs/securesc/thvgrdqa8tpdto4uj02l91ng39l87h8o/fou7dgoi7m0uqhmrhe44me467dbpi1s1/1581795000000/02751230155057180797/12997425790566882683Z/1qrP1Qxrn1LztYOa-qQh_NA44YTGZyZ29?e=downl

In [ ]:
!unzip 'data.zip'

Archive:  data.zip
   creating: data/
   creating: data/test/
   creating: data/test/0/
  inflating: data/test/0/image12.jpg  
  inflating: data/test/0/image124.jpg  
  inflating: data/test/0/image127.jpg  
  inflating: data/test/0/image130.jpg  
  inflating: data/test/0/image150.jpg  
  inflating: data/test/0/image164.jpg  
  inflating: data/test/0/image165.jpg  
  inflating: data/test/0/image166.jpg  
  inflating: data/test/0/image307.jpg  
  inflating: data/test/0/image308.jpg  
  inflating: data/test/0/image315.jpg  
  inflating: data/test/0/image317.jpg  
  inflating: data/test/0/image348.jpg  
  inflating: data/test/0/image349.jpg  
  inflating: data/test/0/image361.jpg  
  inflating: data/test/0/image362.jpg  
  inflating: data/test/0/image363.jpg  
  inflating: data/test/0/image364.jpg  
  inflating: data/test/0/image385.jpg  
  inflating: data/test/0/image483.jpg  
  inflating: data/test/0/image484.jpg  
  inflating: data/test/0/image487.jpg  
  inflating: data/test/0/image492

# New Section

# New Section

In [ ]:
# import matplotlib.image as mpimg
# img = mpimg.imread('ndata/train/1/image83.jpg')
# plt.imshow(img)

In [ ]:
import os, time
import matplotlib.pyplot as plt
import itertools
import pickle
import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms


# G(z)
class generator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(generator, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(100, d*8, 4, 1, 0)
        self.deconv1_bn = nn.BatchNorm2d(d*8)
        self.deconv1_add = nn.ConvTranspose2d(d*8,d*8,3,1,1)
        self.deconv1_add_bn = nn.BatchNorm2d(d*8)
        self.deconv2 = nn.ConvTranspose2d(d*8, d*4, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(d*4)
        self.deconv2_add = nn.ConvTranspose2d(d*4,d*4,3,1,1)
        self.deconv2_add_bn = nn.BatchNorm2d(d*4)
        self.deconv3 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(d*2)
        self.deconv3_add = nn.ConvTranspose2d(d*2,d*2,3,1,1)
        self.deconv3_add_bn = nn.BatchNorm2d(d*2)
        self.deconv4 = nn.ConvTranspose2d(d*2, d, 4, 2, 1)
        self.deconv4_bn = nn.BatchNorm2d(d)
        self.deconv5 = nn.ConvTranspose2d(d, 3, 4, 2, 1)

    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        #x = F.relu(self.deconv1(input))
        x = F.leaky_relu(self.deconv1_bn(self.deconv1(input)), 0.2)
        x = F.leaky_relu(self.deconv1_add_bn(self.deconv1_add(x)), 0.2)
        x = F.leaky_relu(self.deconv2_bn(self.deconv2(x)), 0.2)
        x = F.leaky_relu(self.deconv2_add_bn(self.deconv2_add(x)), 0.2)
        x = F.leaky_relu(self.deconv3_bn(self.deconv3(x)), 0.2)
        x = F.leaky_relu(self.deconv3_add_bn(self.deconv3_add(x)), 0.2)
        x = F.leaky_relu(self.deconv4_bn(self.deconv4(x)), 0.2)
        x = torch.tanh(self.deconv5(x))

        return x

class discriminator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, d, 4, 2, 1)
        #self.conv1 = nn.Conv2d(3,d,3,2,1)
        self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1)
        self.conv2_bn = nn.BatchNorm2d(d*2)
        #self.conv2 = nn.Conv2d(d,d*2,)
        self.conv2_add = nn.Conv2d(d*2,d*2,3,1,1)
        self.conv2_add_bn = nn.BatchNorm2d(d*2)
        self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
        self.conv3_bn = nn.BatchNorm2d(d*4)
        self.conv3_add = nn.Conv2d(d*4,d*4,3,1,1)
        self.conv3_add_bn = nn.BatchNorm2d(d*4)
        self.conv4 = nn.Conv2d(d*4, d*8, 4, 2, 1)
        self.conv4_bn = nn.BatchNorm2d(d*8)
        self.conv4_add = nn.Conv2d(d*8,d*8,3,1,1)
        self.conv4_add_bn = nn.BatchNorm2d(d*8)
        self.conv5 = nn.Conv2d(d*8,1 , 4, 1, 0)
    # weight_init
    def weight_init(self, mean, std):
        for m in self._modules:
            normal_init(self._modules[m], mean, std)

    # forward method
    def forward(self, input):
        x = F.leaky_relu(self.conv1(input), 0.2)
        #print(x.size())
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.conv2_add_bn(self.conv2_add(x)), 0.2)
        #print(x.size())
        x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.conv3_add_bn(self.conv3_add(x)), 0.2)
        #print(x.size())
        x = F.leaky_relu(self.conv4_bn(self.conv4(x)), 0.2)
        x = F.leaky_relu(self.conv4_add_bn(self.conv4_add(x)), 0.2)
        #print(x.size())

        x = torch.sigmoid(self.conv5(x))
        
        return x

def normal_init(m, mean, std):
    if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
        m.weight.data.normal_(mean, std)
        m.bias.data.zero_()

fixed_z_ = torch.randn((5 * 5, 100)).view(-1, 100, 1, 1)    # fixed noise
fixed_z_ = fixed_z_.cuda()
def show_result(num_epoch, show = False, save = False, path = 'result.png', isFix=False):
    z_ = torch.randn((5*5, 100)).view(-1, 100, 1, 1)
    z_ = z_.cuda()

    G.eval()
    if isFix:
        test_images = G(fixed_z_)
    else:
        test_images = G(z_)
    G.train()

    size_figure_grid = 5
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(5, 5))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for k in range(5*5):
          i = k // 5
          j = k % 5
          ax[i, j].cla()
          ax[i, j].imshow(test_images[k, 0].cpu().data.numpy())

    label = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, label, ha='center')
    plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))

    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Iter')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()

# training parameters
batch_size = 64
lr = 0.0002
train_epoch = 50

# data_loader
img_size = 64
transform = transforms.Compose([
          
          transforms.Resize([img_size,img_size]),
          transforms.ToTensor(),
          transforms.Normalize(mean=(0.5,), std=(0.5,))
])
"""train_loader = torch.utils.data.DataLoader(
      datasets.MNIST('data', train=True, download=True, transform=transform),
      batch_size=batch_size, shuffle=True)"""



train_path = "data/train/"

train_dataset = torchvision.datasets.ImageFolder(
    root=train_path,
    transform=transform
)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size, shuffle=False
)
"""train_loader = torchvision.datasets.ImageFolder(
    root=train_path,
    transform=transform
)"""


dataiter = train_loader.__iter__()


images, labels = dataiter.next()



# network
G = generator(64)
D = discriminator(64)
G.weight_init(mean=0.0, std=0.02)
D.weight_init(mean=0.0, std=0.02)
G.cuda()
D.cuda()

# Binary Cross Entropy loss
BCE_loss = nn.BCELoss()

# Adam optimizer
G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
D_optimizer = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

# results save folder
if not os.path.isdir('MNIST_DCGAN_results'):
    os.mkdir('MNIST_DCGAN_results')
if not os.path.isdir('MNIST_DCGAN_results/Random_results'):
    os.mkdir('MNIST_DCGAN_results/Random_results')
if not os.path.isdir('MNIST_DCGAN_results/Fixed_results'):
    os.mkdir('MNIST_DCGAN_results/Fixed_results')

train_hist = {}
train_hist['D_losses'] = []
train_hist['G_losses'] = []
train_hist['per_epoch_ptimes'] = []
train_hist['total_ptime'] = []
num_iter = 0

print('training start!')
start_time = time.time()
for epoch in range(train_epoch):
    D_losses = []
    G_losses = []
    epoch_start_time = time.time()
    for x_, _ in train_loader:
        # train discriminator D
        # print(x_.size())
        D.zero_grad()

        mini_batch = x_.size()[0]

        y_real_ = torch.ones(mini_batch)
        y_fake_ = torch.zeros(mini_batch)

        x_, y_real_, y_fake_ = x_.cuda(), y_real_.cuda(), y_fake_.cuda()
        # print(y_real_.size())
        D_result = D(x_)
        # print(D_result.size())
        D_result= D(x_).squeeze()
        D_real_loss = BCE_loss(D_result, y_real_)

        z_ = torch.randn((mini_batch, 100)).view(-1, 100, 1, 1)
        z_ = z_.cuda()
        G_result = G(z_)

        D_result = D(G_result).squeeze()
        D_fake_loss = BCE_loss(D_result, y_fake_)
        D_fake_score = D_result.data.mean()

        D_train_loss = D_real_loss + D_fake_loss

        D_train_loss.backward()
        D_optimizer.step()

        # D_losses.append(D_train_loss.data[0])
        D_losses.append(D_train_loss.item())

        # train generator G
        G.zero_grad()

        z_ = torch.randn((mini_batch, 100)).view(-1, 100, 1, 1)
        z_ = z_.cuda()

        G_result = G(z_)
        D_result = D(G_result).squeeze()
        G_train_loss = BCE_loss(D_result, y_real_)
        G_train_loss.backward()
        G_optimizer.step()

        G_losses.append(G_train_loss.item())

        num_iter += 1

    epoch_end_time = time.time()
    per_epoch_ptime = epoch_end_time - epoch_start_time


    print('[%d/%d] - ptime: %.2f, loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), train_epoch, per_epoch_ptime, torch.mean(torch.FloatTensor(D_losses)),
                                                              torch.mean(torch.FloatTensor(G_losses))))
    p = 'MNIST_DCGAN_results/Random_results/MNIST_DCGAN_' + str(epoch + 1) + '.png'
    fixed_p = 'MNIST_DCGAN_results/Fixed_results/MNIST_DCGAN_' + str(epoch + 1) + '.png'
    show_result((epoch+1), save=True, path=p, isFix=False)
    show_result((epoch+1), save=True, path=fixed_p, isFix=True)
    train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
    train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))
    train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

end_time = time.time()
total_ptime = end_time - start_time
train_hist['total_ptime'].append(total_ptime)

print("Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f" % (torch.mean(torch.FloatTensor(train_hist['per_epoch_ptimes'])), train_epoch, total_ptime))
print("Training finish!... save training results")
torch.save(G.state_dict(), "MNIST_DCGAN_results/generator_param.pkl")
torch.save(D.state_dict(), "MNIST_DCGAN_results/discriminator_param.pkl")
with open('MNIST_DCGAN_results/train_hist.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

show_train_hist(train_hist, save=True, path='MNIST_DCGAN_results/MNIST_DCGAN_train_hist.png')

images = []
for e in range(train_epoch):
    img_name = 'MNIST_DCGAN_results/Fixed_results/MNIST_DCGAN_' + str(e + 1) + '.png'
    images.append(imageio.imread(img_name))
imageio.mimsave('MNIST_DCGAN_results/generation_animation.gif', images, fps=5)



training start!
[1/50] - ptime: 16.63, loss_d: 1.908, loss_g: 9.682
[2/50] - ptime: 16.57, loss_d: 2.539, loss_g: 16.563
[3/50] - ptime: 16.60, loss_d: 1.725, loss_g: 18.017
[4/50] - ptime: 16.61, loss_d: 1.054, loss_g: 16.968
[5/50] - ptime: 16.76, loss_d: 0.549, loss_g: 9.743
[6/50] - ptime: 16.37, loss_d: 0.913, loss_g: 9.200
[7/50] - ptime: 16.50, loss_d: 0.788, loss_g: 7.759
[8/50] - ptime: 16.24, loss_d: 0.782, loss_g: 4.843
[9/50] - ptime: 16.27, loss_d: 1.151, loss_g: 5.588
[10/50] - ptime: 16.36, loss_d: 1.025, loss_g: 5.580
[11/50] - ptime: 16.44, loss_d: 0.950, loss_g: 5.284
[12/50] - ptime: 16.27, loss_d: 0.665, loss_g: 4.498
[13/50] - ptime: 16.24, loss_d: 1.027, loss_g: 4.668
[14/50] - ptime: 16.40, loss_d: 0.770, loss_g: 4.532
[15/50] - ptime: 16.18, loss_d: 0.568, loss_g: 4.455
[16/50] - ptime: 16.25, loss_d: 0.709, loss_g: 4.262
[17/50] - ptime: 16.40, loss_d: 0.820, loss_g: 4.928
[18/50] - ptime: 16.56, loss_d: 1.108, loss_g: 4.480
[19/50] - ptime: 16.48, loss_d: 0.44